In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
# Read the CSV file
unlabelled_test_data = pd.read_csv("../Dataset_upgrade/unlabelled_test_dataUP.csv")
training_data = pd.read_csv("../Dataset_upgrade/training_dataUP.csv")

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_data['sentence'])
y = training_data['difficulty']

In [24]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Assuming the training data is loaded into a DataFrame named 'training_data'
# with columns 'sentence' and 'difficulty'

# Label encoding the 'difficulty' column
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_data['difficulty'])

# Converting the encoded labels to categorical format
y_categorical = to_categorical(encoded_labels)

# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_data['sentence']).toarray()

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.20, random_state=42)

# Building the Neural Network Model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

# Compiling the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the Model
history = model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

del model

Epoch 1/6
240/240 [==============================] - 1s 2ms/step - loss: 1.5745 - accuracy: 0.3286 - val_loss: 1.3050 - val_accuracy: 0.4594
Epoch 2/6
240/240 [==============================] - 0s 1ms/step - loss: 0.9988 - accuracy: 0.6341 - val_loss: 1.2455 - val_accuracy: 0.4865
Epoch 3/6
240/240 [==============================] - 0s 2ms/step - loss: 0.4767 - accuracy: 0.8555 - val_loss: 1.4450 - val_accuracy: 0.4708
Epoch 4/6
240/240 [==============================] - 1s 2ms/step - loss: 0.2133 - accuracy: 0.9393 - val_loss: 1.7128 - val_accuracy: 0.4656
Epoch 5/6
240/240 [==============================] - 1s 2ms/step - loss: 0.1132 - accuracy: 0.9685 - val_loss: 1.9734 - val_accuracy: 0.4635
Epoch 6/6
30/30 [==============================] - 0s 973us/step - loss: 2.1277 - accuracy: 0.4625
Accuracy: 0.4625000059604645


Let's add more features!

In [26]:
from sklearn.preprocessing import StandardScaler

X_tfidf = vectorizer.fit_transform(training_data['sentence']).toarray()

scaler = StandardScaler()
additional_features = scaler.fit_transform(training_data[['lexical_complexite','sentence_length', 'avg_word_length', 'PUNCT', 'ADV', 'CCONJ', 'X', 'AUX', 'DET', 'PRON', 'NUM', 'NOUN', 'INTJ', 'ADP', 'ADJ', 'VERB', 'PROPN', 'SCONJ']])

# Standardize these features
additional_features_scaled = scaler.fit_transform(additional_features)

# Combine with TF-IDF features
X_combined = np.hstack([X_tfidf, additional_features_scaled])

In [27]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

# Number of folds
num_folds = 5

# K-Fold Cross-Validation
kfold = KFold(n_splits=num_folds, shuffle=True)

# Assuming 'X_combined' is your feature matrix and 'encoded_labels' are your labels
accuracies = []

for train, test in kfold.split(X_combined, encoded_labels):
    # Split data into training and testing sets
    X_train, X_test = X_combined[train], X_combined[test]
    y_train, y_test = encoded_labels[train], encoded_labels[test]

    # Convert labels to categorical
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    # Define the model
    model = Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train_categorical.shape[1], activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train_categorical, epochs=6, batch_size=16)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test_categorical)
    accuracies.append(accuracy)

# Calculate the average and standard deviation of the accuracies
avg_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f"Average accuracy: {avg_accuracy}, Standard Deviation: {std_accuracy}")

del model

Epoch 1/6
216/216 [==============================] - 1s 2ms/step - loss: 1.4120 - accuracy: 0.3898 - val_loss: 1.3295 - val_accuracy: 0.4115
Epoch 2/6
216/216 [==============================] - 0s 2ms/step - loss: 1.0477 - accuracy: 0.5749 - val_loss: 1.2451 - val_accuracy: 0.4609
Epoch 3/6
216/216 [==============================] - 1s 3ms/step - loss: 0.6469 - accuracy: 0.7755 - val_loss: 1.2675 - val_accuracy: 0.4844
Epoch 4/6
216/216 [==============================] - 0s 2ms/step - loss: 0.3168 - accuracy: 0.9144 - val_loss: 1.4385 - val_accuracy: 0.4714
Epoch 5/6
30/30 [==============================] - 0s 1ms/step - loss: 1.5362 - accuracy: 0.4885
Epoch 1/6
216/216 [==============================] - 1s 2ms/step - loss: 1.4417 - accuracy: 0.3718 - val_loss: 1.3289 - val_accuracy: 0.4193
Epoch 2/6
216/216 [==============================] - 0s 1ms/step - loss: 1.0669 - accuracy: 0.5668 - val_loss: 1.2381 - val_accuracy: 0.4609
Epoch 3/6
216/216 [==============================] - 0s 2

In [8]:
from transformers import CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Load tokenizer and model for CamemBERT
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")

# Tokenize and encode sentences in the dataset
inputs = tokenizer(list(training_data['sentence']), padding=True, truncation=True, max_length=128, return_tensors="pt")
input_ids = inputs['input_ids']

# Create a DataLoader for batch processing
batch_size = 8  # Adjust based on your system's capability
dataset = TensorDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Generate embeddings in batches
embeddings = []
model.eval()
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch[0]
        outputs = model(input_ids)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)
embeddings = torch.cat(embeddings, dim=0).numpy()

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [9]:
try:
    del model
    #del embeddings
except:
    print("Model and embeddings already deleted")

In [10]:
import gc
gc.collect()

20

In [11]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=500)
X_text_features = vectorizer.fit_transform(training_data['sentence'])

# Linguistic Features
X_linguistic_features = sp.csr_matrix(training_data[['lexical_complexite', 'sentence_length', 'avg_word_length', 'PUNCT', 'ADV', 'CCONJ', 'X', 'AUX', 'DET', 'PRON', 'NUM', 'NOUN', 'INTJ', 'ADP', 'ADJ', 'VERB', 'PROPN', 'SCONJ']])

# Combine Features
X_combined = sp.hstack([sp.csr_matrix(embeddings), X_text_features, X_linguistic_features])

#Create Interaction Terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_interactions = poly.fit_transform(X_combined.toarray())

#Dimensionality Reduction
pca = PCA(n_components=50)
X_reduced = pca.fit_transform(X_combined.toarray())

# Splitting the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_combined, training_data['difficulty'], test_size=0.2)

In [12]:
from tensorflow.keras import backend as K
K.clear_session()

In [13]:
gc.collect()

0

In [14]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Encode and convert labels to categorical format
label_encoder = LabelEncoder()
y_train_categorical = to_categorical(label_encoder.fit_transform(y_train))
y_test_categorical = to_categorical(label_encoder.transform(y_test))

# Building and compiling the Neural Network Model with added Dropout
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),  # Dropout layer to prevent overfitting
    Dense(64, activation='relu'),
    Dropout(0.5),  # Another Dropout layer
    Dense(y_train_categorical.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train and evaluate the Model with callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Save the best model
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

model.fit(X_train.toarray(), y_train_categorical, epochs=20, batch_size=32, 
          validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

# Evaluate the best model
model.load_weights('best_model.h5')
loss, accuracy = model.evaluate(X_test.toarray(), y_test_categorical)
print(f'Test Accuracy: {accuracy}')


Epoch 1/20
96/96 [==============================] - 0s 2ms/step - loss: 1.8071 - accuracy: 0.2546 - val_loss: 1.5773 - val_accuracy: 0.3607
Epoch 2/20
96/96 [==============================] - 0s 2ms/step - loss: 1.6044 - accuracy: 0.3079 - val_loss: 1.4975 - val_accuracy: 0.3633
Epoch 3/20
96/96 [==============================] - 0s 2ms/step - loss: 1.5110 - accuracy: 0.3542 - val_loss: 1.4403 - val_accuracy: 0.3698
Epoch 4/20
96/96 [==============================] - 0s 1ms/step - loss: 1.4457 - accuracy: 0.3730 - val_loss: 1.3725 - val_accuracy: 0.4023
Epoch 5/20
96/96 [==============================] - 0s 2ms/step - loss: 1.3944 - accuracy: 0.3789 - val_loss: 1.3332 - val_accuracy: 0.4362
Epoch 6/20
96/96 [==============================] - 0s 1ms/step - loss: 1.3558 - accuracy: 0.3936 - val_loss: 1.2897 - val_accuracy: 0.4245
Epoch 7/20
96/96 [==============================] - 0s 1ms/step - loss: 1.3064 - accuracy: 0.4167 - val_loss: 1.2681 - val_accuracy: 0.4297
Epoch 8/20
96/96 [==

In [2]:
import gc

# Define the list of variables to be deleted
variables = [
    "model",
    "embeddings",
    "X_text_features",
    "X_linguistic_features",
    "X_combined",
    "X_interactions",
    "X_reduced",
    "X_train",
    "X_test",
    "y_train",
    "y_test",
    "y_train_categorical",
    "y_test_categorical",
    "label_encoder",
    "dataset",
    "dataloader",
    "input_ids",
    "inputs",
    "batch",
    "batch_embeddings",
    "outputs",
    "tokenizer",
    "vectorizer",
    "poly",
    "pca",
    "early_stopping",
    "model_checkpoint",
    "loss",
    "accuracy",
]

# Iterate over the list of variables and delete them
for variable in variables:
    if variable in globals():
        try:
            del globals()[variable]
        except NameError:
            pass

# Collect garbage to free up memory
gc.collect()
del gc
